In [20]:
%matplotlib inline

In [313]:
import plotly.plotly as py
py.sign_in('your_login', 'your_api_key')

from plotly.graph_objs import *

In [3]:
import pandas as pd
from six import print_

Подключаемся к SQLite3-базе

In [ ]:
from sqlalchemy import create_engine

eng = create_engine("sqlite:///service311.db")
conn = eng.connect()

Пример одной строки из таблицы `service311`

In [106]:
pd.read_sql_query("SELECT * FROM service311 LIMIT 1;", conn)

,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,...,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Garage Lot Name,Ferry Direction,Ferry Terminal Name,Latitude,Longitude,Location
0,31015465,07/06/2015 10:58:27 AM,07/22/2015 01:07:20 AM,DCA,Department of Consumer Affairs,Consumer Complaint,Demand for Cash,,11360.0,27-16 203 STREET,...,,,,,,,,40.77354,-73.788237,"(40.773539552542, -73.78823697228408)"


Список всех колонок

In [105]:
list(pd.read_sql_query("SELECT * FROM service311 LIMIT 1;", conn).columns)

[u'Unique Key',
 u'Created Date',
 u'Closed Date',
 u'Agency',
 u'Agency Name',
 u'Complaint Type',
 u'Descriptor',
 u'Location Type',
 u'Incident Zip',
 u'Incident Address',
 u'Street Name',
 u'Cross Street 1',
 u'Cross Street 2',
 u'Intersection Street 1',
 u'Intersection Street 2',
 u'Address Type',
 u'City',
 u'Landmark',
 u'Facility Type',
 u'Status',
 u'Due Date',
 u'Resolution Description',
 u'Resolution Action Updated Date',
 u'Community Board',
 u'Borough',
 u'X Coordinate (State Plane)',
 u'Y Coordinate (State Plane)',
 u'Park Facility Name',
 u'Park Borough',
 u'School Name',
 u'School Number',
 u'School Region',
 u'School Code',
 u'School Phone Number',
 u'School Address',
 u'School City',
 u'School State',
 u'School Zip',
 u'School Not Found',
 u'School or Citywide Complaint',
 u'Vehicle Type',
 u'Taxi Company Borough',
 u'Taxi Pick Up Location',
 u'Bridge Highway Name',
 u'Bridge Highway Direction',
 u'Road Ramp',
 u'Bridge Highway Segment',
 u'Garage Lot Name',
 u'Ferry 

Получите из базы ТОП-10 самых популярных по количеству обращений в службу 311 городов (без учета регистра) в районе Нью Йорка.

In [212]:
cities = pd.read_sql_query("YOUR SQL QUERY", conn)

data = [Bar(x=cities['city'], y=cities['counts'])]
py.iplot(data)

# Ниже ожидаемый результат

In [5]:
### YOUR CODE

Возьмите ТОП-3 городов из пункта выше, посчитайте для каждого из них количество вызовов в каждый из дней и нарисуйте их на графике. С форматированием дат придется повозитьcя :-)

In [222]:
dates = {}

for city in cities.city[:3]:
    df = pd.read_sql_query("""
        YOUR SQL QUERY
    """.format(city=city), conn)
    dates[city] = df
    
data = [Scatter(x=df['created_at'],y=df['counts'], name=city) for city, df in dates.items()]
py.iplot(data)

# Ожидаемый результат

Сделайте аналогичный график без plot.ly по аналогии с http://playittodeath.ru/Exploratory_graphs.html

In [7]:
### YOUR CODE HERE

Найдите несколько самых популярных разновидностей жалоб (`Descriptor`)

In [4]:
### YOUR CODE

Загрузите в объект `df` поля `Latitude` и `Longitude` тех строк, у которых координаты указаны (`<>''`), поле `Descriptor` равно `Loud Music/Party` и дата с 7 по 9 января 2015 года включительно.

In [308]:
df = pd.read_sql_query("""
        YOUR SQL Query;
    """, conn)

In [310]:
points = ", ".join("[{},{}]".format(lat, lon) for lat, lon in zip(df.Latitude, df.Longitude))

In [311]:
clusters_js = """

ymaps.ready(function () {{
    var myMap = new ymaps.Map('map', {{
            center: [{}, {}],
            zoom: 9,
            behaviors: ['default', 'scrollZoom']
        }}, {{
            searchControlProvider: 'yandex#search'
        }}),
        /**
         * Создадим кластеризатор, вызвав функцию-конструктор.
         * Список всех опций доступен в документации.
         * @see https://api.yandex.ru/maps/doc/jsapi/2.1/ref/reference/Clusterer.xml#constructor-summary
         */
            clusterer = new ymaps.Clusterer({{
            /**
             * Через кластеризатор можно указать только стили кластеров,
             * стили для меток нужно назначать каждой метке отдельно.
             * @see https://api.yandex.ru/maps/doc/jsapi/2.1/ref/reference/option.presetStorage.xml
             */
            preset: 'islands#invertedVioletClusterIcons',
            /**
             * Ставим true, если хотим кластеризовать только точки с одинаковыми координатами.
             */
            groupByCoordinates: false,
            /**
             * Опции кластеров указываем в кластеризаторе с префиксом "cluster".
             * @see https://api.yandex.ru/maps/doc/jsapi/2.1/ref/reference/ClusterPlacemark.xml
             */
            clusterDisableClickZoom: true,
            clusterHideIconOnBalloonOpen: false,
            geoObjectHideIconOnBalloonOpen: false
        }}),
        /**
         * Функция возвращает объект, содержащий данные метки.
         * Поле данных clusterCaption будет отображено в списке геообъектов в балуне кластера.
         * Поле balloonContentBody - источник данных для контента балуна.
         * Оба поля поддерживают HTML-разметку.
         * Список полей данных, которые используют стандартные макеты содержимого иконки метки
         * и балуна геообъектов, можно посмотреть в документации.
         * @see https://api.yandex.ru/maps/doc/jsapi/2.1/ref/reference/GeoObject.xml
         */
            getPointData = function (index) {{
            return {{
                balloonContentBody: 'балун <strong>метки ' + index + '</strong>',
                clusterCaption: 'метка <strong>' + index + '</strong>'
            }};
        }},
        /**
         * Функция возвращает объект, содержащий опции метки.
         * Все опции, которые поддерживают геообъекты, можно посмотреть в документации.
         * @see https://api.yandex.ru/maps/doc/jsapi/2.1/ref/reference/GeoObject.xml
         */
            getPointOptions = function () {{
            return {{
                preset: 'islands#violetIcon'
            }};
        }},
        points = [
            {}
        ],
        geoObjects = [];

    /**
     * Данные передаются вторым параметром в конструктор метки, опции - третьим.
     * @see https://api.yandex.ru/maps/doc/jsapi/2.1/ref/reference/Placemark.xml#constructor-summary
     */
    for(var i = 0, len = points.length; i < len; i++) {{
        geoObjects[i] = new ymaps.Placemark(points[i], getPointData(i), getPointOptions());
    }}

    /**
     * Можно менять опции кластеризатора после создания.
     */
    clusterer.options.set({{
        gridSize: 80,
        clusterDisableClickZoom: true
    }});

    /**
     * В кластеризатор можно добавить javascript-массив меток (не геоколлекцию) или одну метку.
     * @see https://api.yandex.ru/maps/doc/jsapi/2.1/ref/reference/Clusterer.xml#add
     */
    clusterer.add(geoObjects);
    myMap.geoObjects.add(clusterer);

    /**
     * Спозиционируем карту так, чтобы на ней были видны все объекты.
     */

    myMap.setBounds(clusterer.getBounds(), {{
        checkZoomRange: true
    }});
}});

""".format(df.Latitude.mean(), df.Longitude.mean(), points)

In [312]:
with open('clusters.js', 'w') as fout:
    print_(clusters_js, file=fout)

Откройте Anaconda Promt и командой `python -m SimpleHTTPServer` запустите встроенный Python-вебсервер. Откройте `http://localhost:8000/clusters.html` и посмотрите, что получилось.

Придумайте еще интересные визуализации этого набора данных (1-2) с помощью библиотек Plot.ly, Seaborn или других.

In [6]:
### YOUR CODE HERE